In [ ]:
# pip install pycaret

In [ ]:
# pip install markupsafe==2.0.1

##Importing dependencies

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import lightgbm
from pickle import dump
from sklearn.preprocessing import RobustScaler
from pycaret.regression import *

In [48]:
np.random.seed(13)

##Preprocessing Function

In [49]:
def preprocess(df):
  df.fillna(0)
  col=df.columns
  df[col[1]].replace(['RK1','BHK1','BHK2','BHK3','BHK4','BHK4PLUS'],[0,1,2,3,4,5],inplace=True)
  df[col[3]] = pd.to_datetime(df[col[3]])
  df['activation_year'] = df['activation_date'].dt.year
  df['activation_month'] = df['activation_date'].dt.month
  df['activation_day'] = df['activation_date'].dt.day
  df['activation_hour'] = df['activation_date'].dt.hour
  df['activation_minute'] = df['activation_date'].dt.minute
  df[col[6]].replace(['FAMILY','ANYONE','BACHELOR','COMPANY'],[0,1,2,3],inplace=True)
  df[col[11]].replace(['SEMI_FURNISHED','FULLY_FURNISHED','NOT_FURNISHED'],[1,2,0],inplace=True)
  df[col[12]].replace(['BOTH','TWO_WHEELER','NONE','FOUR_WHEELER'],[3,1,0,2],inplace=True)
  df[col[14]] = df[col[14]].clip(lower=0)
  df[col[16]].replace(['E', 'NE', 'S', 'N', 'SE', 'W', 'NW', 'SW'],[0,1,2,3,4,5,6,7],inplace=True)
  df[col[21]].replace(['CORP_BORE', 'CORPORATION', 'BOREWELL'],[0,1,2],inplace=True)
  df[col[22]].replace(['AP', 'IH', 'IF', 'GC'],[0,1,2,3],inplace=True)


  def str_to_dict(string):
    res=json.loads(string)
    return res


  df[col[20]]=df[col[20]].map(str_to_dict)
  attri=df[col[20]].apply(pd.Series)
  
  # Working on attributes
  newcol=attri.columns


  def as_int_type(column):
    return column.fillna(0).astype(int)


  for i in range(19):
    attri[newcol[i]]=as_int_type(attri[newcol[i]])

  df.drop([col[0],col[2],col[3],col[20]],axis=1,inplace=True)
  predata = pd.concat([df, attri], axis=1)
  return predata

In [77]:
dftrain=pd.read_csv('train.csv')
dftest=pd.read_csv('test.csv')

Target feature extracted

In [78]:
target=dftrain['rent']
dftrain.drop(['rent'],axis=1,inplace=True)
pretrain = preprocess(dftrain)

In [79]:
pretest = preprocess(dftest)

In [80]:
pretrain.describe()

,type,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,...,SERVANT,SECURITY,SC,GP,PARK,RWH,STP,HK,PB,VP
count,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,...,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000,20500.000000
mean,2.012439,12.945751,77.637280,0.538000,0.226732,0.361463,0.174927,0.711512,0.987659,2.073366,...,0.037268,0.473561,0.471122,0.100098,0.505366,0.247024,0.171024,0.198732,0.365610,0.238878
std,0.746517,0.029594,0.110586,0.568169,0.418728,0.480436,0.379914,0.453070,0.356268,1.037557,...,0.189423,0.499313,0.499178,0.300137,0.499983,0.431292,0.376539,0.399055,0.481612,0.426408
min,0.000000,12.900004,77.500072,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,12.918188,77.571956,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,12.943800,77.633491,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,12.970952,77.695760,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,3.000000,...,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,5.000000,12.999999,80.266346,3.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [59]:
# pretrain.to_csv('out.csv',index=False) 

##Robust Scaling for transformation

In [82]:
transformer = RobustScaler().fit(pretrain)
scaletrain=pd.DataFrame(transformer.transform(pretrain))

In [ ]:
scaletrain.describe()

Storing the transformer in pickle

In [22]:
dump(transformer, open('robust_scaler.pkl', 'wb'))

In [85]:
scaletest=pd.DataFrame(transformer.transform(pretest))

Removing features with correlation >= 0.85

In [86]:
cor = scaletrain.corr()
columns = np.full(cor.shape[0], True, dtype=bool)
for i in range(cor.shape[0]):
    for j in range(i+1, cor.shape[0]):
        if cor.iloc[i,j] >= 0.85:
            if columns[j]:
                columns[j] = False

In [87]:
selected_columns = scaletrain.columns[columns]
selected_columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 32, 33, 34, 35, 36,
            37, 38, 39, 40, 41, 42, 43],
           dtype='int64')

In [91]:
cortrain=scaletrain[selected_columns]

In [ ]:
cortrain.info()

In [89]:
cortest=scaletest[selected_columns]

In [92]:
cortrain['target']=target

##Using pycaret to find the best model


In [ ]:
exp = setup(data=cortrain,target='target')

In [ ]:
best_model=compare_models(sort='RMSE')

In [ ]:
best_tune=tune_model(best_model)

## Light Gradient Boosting Machine gives the best RMSE and R2

In [93]:
model=lightgbm.LGBMRegressor(bagging_fraction=0.6, bagging_freq=4, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=11, min_child_weight=0.001, min_split_gain=0.5,
              n_estimators=270, n_jobs=-1, num_leaves=90, objective=None,
              random_state=7898, reg_alpha=2, reg_lambda=5, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [94]:
model_pkl=model.fit(cortrain.iloc[:, :-1], cortrain.iloc[:, -1])

Storing trained model in pickle

In [99]:
dump(model_pkl, open('lgbm_model.pkl', 'wb'))

In [98]:
submit=pd.DataFrame()
submit['rent']=model_pkl.predict(cortest)
submit.to_csv('submission.csv',index=False,)

Deployed on github
https://github.com/roaringsingh


Submitted By: Prabhnoor Singh
